# Convolutonal Neural Network Using pretrained model

In [1]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16

In [2]:
conv_base = VGG16(include_top = False, weights='imagenet' , input_shape = (150,150,3))   # shape of imageNet

In [3]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [4]:
Base_dir = "C:/Users/Dell/Desktop/pac/Quater 3/Project/Try-3/Folders_Of_Picture"
train_dir = os.path.join(Base_dir,'train')
test_dir = os.path.join(Base_dir,"test")
Validation_dir = os.path.join(Base_dir,"Validation")

In [5]:
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

In [6]:
def feature_extraction(directory,sample_count):
    features = np.zeros(shape=(sample_count,4,4,512))
    labels = np.zeros(shape = (sample_count,len(os.listdir(directory))))
    data_generator = datagen.flow_from_directory(directory,target_size=(150,150),batch_size = batch_size , class_mode = "categorical")
    i = 0
    for input_batch, label_batch in data_generator:
        features_batch = conv_base.predict(input_batch)
        features[i * batch_size : (i+1) * batch_size] = features_batch
        labels[i * batch_size : (i+1) * batch_size] = label_batch
        i += 1
        if i* batch_size >=sample_count:
            break
        
    return features, labels
    

In [7]:
train_feature,train_labels = feature_extraction(train_dir , 12)
test_feature,test_labels = feature_extraction(test_dir , 6)
Validation_feature,Validation_labels = feature_extraction(Validation_dir , 6)

Found 12 images belonging to 3 classes.
Found 6 images belonging to 3 classes.
Found 6 images belonging to 3 classes.


In [8]:
train_feature = np.reshape(train_feature , (2000, 12,4*4*512))
test_feature = np.reshape(test_feature , (1000,6,4*4*512))
Validation_feature = np.reshape(Validation_feature , (1000, 6,4*4*512))

ValueError: cannot reshape array of size 98304 into shape (2000,12,8192)

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import models

model = models.Sequential()

model.add(layers.Dense(512, activation='relu' , input_shape = (4*4*512,)))
model.add(layers.Dense(3, activation='softmax'))

In [ ]:
model.compile(loss = "categorical_crossentropy" , optimizer = "rmsprop" , metrics=["acc"])

In [ ]:
history = model.fit(train_feature,train_labels,epochs=5,batch_size = 20)   #validation = (validation_features , validation_labels )

#  validation data can be added by this form but on project try 3 , there is no validation data in fit so to compare accuracy we will keep it similar 

In [ ]:
model.evaluate(test_feature,test_labels)

In [ ]:
#   acurracy increased from 33.4% to 50%


#   The accuracy does not increased that much because we use imagenet for humman  we should use facenet instead